In [1]:
import sys
sys.path.insert(0, 'Utilities/')
import os

from scipy.interpolate import griddata
from pyDOE import lhs
from plotting import newfig, savefig
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.io

2024-04-25 15:19:22.503150: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
np.random.seed(seed=1234)
tf.random.set_seed(1234)
tf.config.experimental.enable_tensor_float_32_execution(False)
#os.environ[‘TF_ENABLE_AUTO_MIXED_PRECISION’] = ‘1’

In [3]:
# Initalization of Network
def hyper_initial(size):
    in_dim = size[0]
    out_dim = size[1]
    std = np.sqrt(2.0/(in_dim + out_dim))
    return tf.Variable(tf.random.truncated_normal(shape=size, stddev = std))

# Neural Network 
def DNN(X, W, b):
    A = X
    L = len(W)
    for i in range(L-1):
        A = tf.tanh(tf.add(tf.matmul(A, W[i]), b[i]))
    Y = tf.add(tf.matmul(A, W[-1]), b[-1])
    return Y

def train_vars(W, b):
    return W + b 

def net_u(x, t, w, b):
    u = DNN(tf.concat([x,t],1), w, b)
    return u


#@tf.function(jit_compile=True)
@tf.function()
def net_f(x,t,W, b, nu):
    with tf.GradientTape(persistent=True) as tape1:
        tape1.watch([x, t])
        with tf.GradientTape(persistent=True) as tape2:
            tape2.watch([x, t])
            u=net_u(x,t, W, b)
        u_t = tape2.gradient(u, t)
        u_x = tape2.gradient(u, x)
    u_xx = tape1.gradient(u_x, x)  
    f = u_t + u*u_x - nu*u_xx
    return f


#@tf.function()
@tf.function()
def train_step(W, b, X_u_train_tf, u_train_tf, X_f_train_tf, opt, nu):
    x_u = X_u_train_tf[:,0:1]
    t_u = X_u_train_tf[:,1:2]
    x_f = X_f_train_tf[:,0:1]
    t_f = X_f_train_tf[:,1:2]
    with tf.GradientTape() as tape:
        tape.watch([W,b])
        u_nn = net_u(x_u, t_u, W, b) 
        f_nn = net_f(x_f,t_f, W, b, nu)
        loss =  tf.reduce_mean(tf.square(u_nn - u_train_tf)) +\
        tf.reduce_mean(tf.square(f_nn)) 
    grads = tape.gradient(loss, train_vars(W, b))
    opt.apply_gradients(zip(grads, train_vars(W,b)))
    return loss

In [27]:
nu = 0.01/np.pi # Viscosity
N_u = 100 # Number of Initial and Boundary data points
N_f = 10000 # Number of residual point
Nmax=  20000

layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]

L = len(layers)
W = [hyper_initial([layers[l-1], layers[l]]) for l in range(1, L)] 
b = [tf.Variable(tf.zeros([1, layers[l]])) for l in range(1, L)]
u_star = Exact.flatten()[:,None]

data = scipy.io.loadmat('./Data/burgers_shock.mat')
t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T
X, T = np.meshgrid(x,t)
X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact.flatten()[:,None]

# Doman bounds
lb = X_star.min(0)
ub = X_star.max(0)  

# Initial Condition
xx1 = np.hstack((X[0:1,:].T, T[0:1,:].T))
uu1 = Exact[0:1,:].T

# Boundary condition -1
xx2 = np.hstack((X[:,0:1], T[:,0:1]))
uu2 = Exact[:,0:1]

# Boundary condition 1
xx3 = np.hstack((X[:,-1:], T[:,-1:]))
uu3 = Exact[:,-1:]

X_u_train = np.vstack([xx1, xx2, xx3])
X_f_train = lb + (ub-lb)*lhs(2, N_f)
X_f_train = np.vstack((X_f_train, X_u_train))
u_train = np.vstack([uu1, uu2, uu3])

In [31]:
np.vstack([xx1, xx2, xx3])

array([[-1.        ,  0.        ],
       [-0.99215686,  0.        ],
       [-0.98431373,  0.        ],
       [-0.97647059,  0.        ],
       [-0.96862745,  0.        ],
       [-0.96078431,  0.        ],
       [-0.95294118,  0.        ],
       [-0.94509804,  0.        ],
       [-0.9372549 ,  0.        ],
       [-0.92941176,  0.        ],
       [-0.92156863,  0.        ],
       [-0.91372549,  0.        ],
       [-0.90588235,  0.        ],
       [-0.89803922,  0.        ],
       [-0.89019608,  0.        ],
       [-0.88235294,  0.        ],
       [-0.8745098 ,  0.        ],
       [-0.86666667,  0.        ],
       [-0.85882353,  0.        ],
       [-0.85098039,  0.        ],
       [-0.84313725,  0.        ],
       [-0.83529412,  0.        ],
       [-0.82745098,  0.        ],
       [-0.81960784,  0.        ],
       [-0.81176471,  0.        ],
       [-0.80392157,  0.        ],
       [-0.79607843,  0.        ],
       [-0.78823529,  0.        ],
       [-0.78039216,

In [33]:
lb + (ub-lb)*lhs(2, N_f)

array([[ 0.96102861,  0.20307343],
       [ 0.1495052 ,  0.84258634],
       [-0.1432034 ,  0.45354875],
       ...,
       [ 0.94686801,  0.05440626],
       [-0.18494551,  0.73339428],
       [ 0.77829727,  0.40880675]])

In [16]:
X.flatten()[:,None]

array([[-1.        ],
       [-0.99215686],
       [-0.98431373],
       ...,
       [ 0.98431373],
       [ 0.99215686],
       [ 1.        ]])

In [12]:
T.flatten()[:,None]

array([[0.  ],
       [0.  ],
       [0.  ],
       ...,
       [0.99],
       [0.99],
       [0.99]])

In [33]:
np.hstack((X.flatten()[:,None], T.flatten()[:,None]))

array([[-1.        ,  0.        ],
       [-0.99215686,  0.        ],
       [-0.98431373,  0.        ],
       ...,
       [ 0.98431373,  0.99      ],
       [ 0.99215686,  0.99      ],
       [ 1.        ,  0.99      ]])

In [37]:
Exact.flatten()[:, None]

array([[ 1.22464680e-16],
       [ 2.46374492e-02],
       [ 4.92599411e-02],
       ...,
       [-1.19673204e-02],
       [-5.98368729e-03],
       [ 1.12388795e-16]])

In [18]:
X_star.min(0)

array([-1.,  0.])

In [61]:
# Doman bounds
lb = X_star.min(0)
ub = X_star.max(0)  
# Initial Condition
xx1 = np.hstack((X[0:1,:].T, T[0:1,:].T))
uu1 = Exact[0:1,:].T
# Boundary condition -1
xx2 = np.hstack((X[:,0:1], T[:,0:1]))
uu2 = Exact[:,0:1]
# Boundary condition 1
xx3 = np.hstack((X[:,-1:], T[:,-1:]))
uu3 = Exact[:,-1:]
X_u_train = np.vstack([xx1, xx2, xx3])
X_f_train = lb + (ub-lb)*lhs(2, N_f)
X_f_train = np.vstack((X_f_train, X_u_train))
u_train = np.vstack([uu1, uu2, uu3])

In [60]:
np.vstack([xx1, xx2, xx3])

array([[-1.        ,  0.        ],
       [-0.99215686,  0.        ],
       [-0.98431373,  0.        ],
       [-0.97647059,  0.        ],
       [-0.96862745,  0.        ],
       [-0.96078431,  0.        ],
       [-0.95294118,  0.        ],
       [-0.94509804,  0.        ],
       [-0.9372549 ,  0.        ],
       [-0.92941176,  0.        ],
       [-0.92156863,  0.        ],
       [-0.91372549,  0.        ],
       [-0.90588235,  0.        ],
       [-0.89803922,  0.        ],
       [-0.89019608,  0.        ],
       [-0.88235294,  0.        ],
       [-0.8745098 ,  0.        ],
       [-0.86666667,  0.        ],
       [-0.85882353,  0.        ],
       [-0.85098039,  0.        ],
       [-0.84313725,  0.        ],
       [-0.83529412,  0.        ],
       [-0.82745098,  0.        ],
       [-0.81960784,  0.        ],
       [-0.81176471,  0.        ],
       [-0.80392157,  0.        ],
       [-0.79607843,  0.        ],
       [-0.78823529,  0.        ],
       [-0.78039216,

In [25]:
lb + (ub-lb)*lhs(2, N_f)

array([[ 0.35794977,  0.17239227],
       [ 0.92892385,  0.85446821],
       [ 0.81462871,  0.18926602],
       ...,
       [ 0.04650701,  0.25705145],
       [ 0.05127784,  0.17229916],
       [-0.67942849,  0.40419442]])

In [26]:
np.vstack((X_f_train, X_u_train))

NameError: name 'X_f_train' is not defined

In [20]:
lb = X_star.min(0)
ub = X_star.max(0)  